In [8]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time

from sklearn.model_selection import StratifiedKFold
device = torch.device('cuda')

In [9]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'botnet26t_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_xlarge_in22k',
 'crossvit_9_240',
 'crossv

In [10]:
path = '/home/'

In [11]:
train_png = sorted(glob(path + 'open/train/*.png'))
test_png = sorted(glob(path + 'open/test/*.png'))

In [12]:
len(train_png), len(test_png)

(4277, 2154)

In [13]:
train_y = pd.read_csv(path +"open/train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [14]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (384, 384),interpolation = cv2.INTER_AREA)
    return img

In [15]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|██████████| 2154/2154 [01:25<00:00, 25.14it/s]


In [16]:
np.save(path + 'train_imgs_384', np.array(train_imgs))
np.save(path + 'test_imgs_384', np.array(test_imgs))

In [17]:
train_imgs = np.load(path + 'train_imgs_384.npy')
test_imgs = np.load(path + 'test_imgs_384.npy')

In [18]:
meanRGB = [np.mean(x, axis=(0,1)) for x in train_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in train_imgs]

meanR = np.mean([m[0] for m in meanRGB])/255
meanG = np.mean([m[1] for m in meanRGB])/255
meanB = np.mean([m[2] for m in meanRGB])/255

stdR = np.mean([s[0] for s in stdRGB])/255
stdG = np.mean([s[1] for s in stdRGB])/255
stdB = np.mean([s[2] for s in stdRGB])/255

print("train 평균",meanR, meanG, meanB)
print("train 표준편차",stdR, stdG, stdB)

train 평균 0.4330380901867049 0.4034575319032911 0.39415050509784405
train 표준편차 0.1815717110252788 0.17403455556798705 0.16323395055036488


In [19]:
meanRGB = [np.mean(x, axis=(0,1)) for x in test_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in test_imgs]

meanR = np.mean([m[0] for m in meanRGB])/255
meanG = np.mean([m[1] for m in meanRGB])/255
meanB = np.mean([m[2] for m in meanRGB])/255

stdR = np.mean([s[0] for s in stdRGB])/255
stdG = np.mean([s[1] for s in stdRGB])/255
stdB = np.mean([s[2] for s in stdRGB])/255

print("test 평균",meanR, meanG, meanB)
print("test 표준편차",stdR, stdG, stdB)

test 평균 0.41825619520929724 0.3931011906330291 0.386631764639131
test 표준편차 0.19505524270747931 0.19005280951759498 0.18053225852732663


In [24]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode == 'train':
          train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.433038, 0.403458, 0.394151],
                                     std = [0.181572, 0.174035, 0.163234]),
                transforms.RandomAffine((-45, 45)),
                
            ])
          img = train_transform(img)
        if self.mode == 'test':
          test_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.418256, 0.393101, 0.386632],
                                     std = [0.195055, 0.190053, 0.185323])
            ])
          img = test_transform(img)

        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self,mode = 'train'):
        super(Network, self).__init__()
        self.mode = mode
        if self.mode == 'train':
          self.model = timm.create_model('densenet121', pretrained=True, num_classes=88)
        if self.mode == 'test':
          self.model = timm.create_model('densenet121', pretrained=True, num_classes=88)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [25]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [26]:
def main(seed = 2022):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
    
main(2022)

In [28]:
import gc

cv = StratifiedKFold(n_splits = 5, random_state = 2022,shuffle=True)
batch_size = 16
epochs = 100
pred_ensemble = []


for idx, (train_idx, val_idx) in enumerate(cv.split(train_imgs, np.array(train_labels))):
  print("----------fold_{} start!----------".format(idx))
  t_imgs, val_imgs = train_imgs[train_idx],  train_imgs[val_idx]
  t_labels, val_labels = np.array(train_labels)[train_idx], np.array(train_labels)[val_idx]

  # Train
  train_dataset = Custom_dataset(np.array(t_imgs), np.array(t_labels), mode='train')
  train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

  # Val
  val_dataset = Custom_dataset(np.array(val_imgs), np.array(val_labels), mode='test')
  val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)

  gc.collect()
  torch.cuda.empty_cache()
  best=0

  model = Network().to(device)

  optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay = 1e-3)
  criterion = nn.CrossEntropyLoss()
  scaler = torch.cuda.amp.GradScaler()  

  best_f1 = 0
  early_stopping = 0
  for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
    train_f1 = score_function(train_y, train_pred)
    state_dict= model.state_dict()
    model.eval()
    with torch.no_grad():
      val_loss = 0 
      val_pred = []
      val_y = []
      

      for batch in (val_loader):
        x_val = torch.tensor(batch[0], dtype = torch.float32, device = device)
        y_val = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred_val = model(x_val)
        loss_val = criterion(pred_val, y_val)

        val_loss += loss_val.item()/len(val_loader)
        val_pred += pred_val.argmax(1).detach().cpu().numpy().tolist()
        val_y += y_val.detach().cpu().numpy().tolist()
      val_f1 = score_function(val_y, val_pred)

      if val_f1 > best_f1:
        best_epoch = epoch
        best_loss = val_loss
        best_f1 = val_f1
        early_stopping = 0

        torch.save({'epoch':epoch,
                    'state_dict':state_dict,
                    'optimizer': optimizer.state_dict(),
                    'scaler': scaler.state_dict(),
             }, path +'best_model_{}.pth'.format(idx))
        print('-----------------SAVE:{} epoch----------------'.format(best_epoch+1))
      else:
          early_stopping += 1

            # Early Stopping
      if early_stopping == 20:
        TIME = time.time() - start
        print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
        print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
        print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')
        break

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')

----------fold_0 start!----------
-----------------SAVE:1 epoch----------------
epoch : 1/100    time : 55s/5402s
TRAIN    loss : 1.39541    f1 : 0.15106
Val    loss : 0.79334    f1 : 0.15600
-----------------SAVE:2 epoch----------------
epoch : 2/100    time : 54s/5340s
TRAIN    loss : 0.75454    f1 : 0.17218
Val    loss : 0.60800    f1 : 0.21792
-----------------SAVE:3 epoch----------------
epoch : 3/100    time : 54s/5281s
TRAIN    loss : 0.56252    f1 : 0.24395
Val    loss : 0.45816    f1 : 0.29586
-----------------SAVE:4 epoch----------------
epoch : 4/100    time : 56s/5389s
TRAIN    loss : 0.44076    f1 : 0.37636
Val    loss : 0.39546    f1 : 0.37855
-----------------SAVE:5 epoch----------------
epoch : 5/100    time : 54s/5172s
TRAIN    loss : 0.36290    f1 : 0.49377
Val    loss : 0.32091    f1 : 0.42818
-----------------SAVE:6 epoch----------------
epoch : 6/100    time : 54s/5112s
TRAIN    loss : 0.28459    f1 : 0.60304
Val    loss : 0.24839    f1 : 0.58239
-----------------S

epoch : 67/100    time : 54s/1773s
TRAIN    loss : 0.01530    f1 : 0.97891
Val    loss : 0.26976    f1 : 0.73426
epoch : 68/100    time : 55s/1751s
TRAIN    loss : 0.01211    f1 : 0.98675
Val    loss : 0.20046    f1 : 0.79940
epoch : 69/100    time : 53s/1644s
TRAIN    loss : 0.00713    f1 : 0.99479
Val    loss : 0.20705    f1 : 0.77863
epoch : 70/100    time : 56s/1677s
TRAIN    loss : 0.01943    f1 : 0.98352
Val    loss : 0.22508    f1 : 0.77000
epoch : 71/100    time : 54s/1576s
TRAIN    loss : 0.02980    f1 : 0.98021
Val    loss : 0.23807    f1 : 0.75186
epoch : 72/100    time : 53s/1496s
TRAIN    loss : 0.01711    f1 : 0.99013
Val    loss : 0.25268    f1 : 0.75083
epoch : 73/100    time : 56s/1515s
TRAIN    loss : 0.03646    f1 : 0.96078
Val    loss : 0.20223    f1 : 0.72375
epoch : 74/100    time : 54s/1403s
TRAIN    loss : 0.03837    f1 : 0.95584
Val    loss : 0.16735    f1 : 0.79409
epoch : 75/100    time : 54s/1340s
TRAIN    loss : 0.01311    f1 : 0.99382
Val    loss : 0.16362

epoch : 36/100    time : 53s/3422s
TRAIN    loss : 0.01708    f1 : 0.98892
Val    loss : 0.18421    f1 : 0.81015
epoch : 37/100    time : 54s/3411s
TRAIN    loss : 0.01789    f1 : 0.97784
Val    loss : 0.17034    f1 : 0.78908
epoch : 38/100    time : 57s/3527s
TRAIN    loss : 0.02513    f1 : 0.97257
Val    loss : 0.18125    f1 : 0.79018
epoch : 39/100    time : 52s/3196s
TRAIN    loss : 0.03360    f1 : 0.97592
Val    loss : 0.27344    f1 : 0.72489
-----------------SAVE:40 epoch----------------
epoch : 40/100    time : 53s/3184s
TRAIN    loss : 0.02829    f1 : 0.97625
Val    loss : 0.19035    f1 : 0.81585
-----------------SAVE:41 epoch----------------
epoch : 41/100    time : 53s/3107s
TRAIN    loss : 0.01630    f1 : 0.98812
Val    loss : 0.22086    f1 : 0.81938
-----------------SAVE:42 epoch----------------
epoch : 42/100    time : 55s/3193s
TRAIN    loss : 0.03724    f1 : 0.95699
Val    loss : 0.16691    f1 : 0.83111
epoch : 43/100    time : 53s/3014s
TRAIN    loss : 0.01345    f1 : 0

epoch : 32/100    time : 54s/3654s
TRAIN    loss : 0.05065    f1 : 0.95382
Val    loss : 0.23538    f1 : 0.74777
epoch : 33/100    time : 56s/3731s
TRAIN    loss : 0.03524    f1 : 0.95800
Val    loss : 0.16378    f1 : 0.81472
epoch : 34/100    time : 53s/3475s
TRAIN    loss : 0.03060    f1 : 0.96301
Val    loss : 0.42426    f1 : 0.72920
epoch : 35/100    time : 54s/3508s
TRAIN    loss : 0.03032    f1 : 0.97879
Val    loss : 0.18545    f1 : 0.78227
epoch : 36/100    time : 56s/3557s
TRAIN    loss : 0.03316    f1 : 0.97386
Val    loss : 0.17114    f1 : 0.78257
epoch : 37/100    time : 54s/3396s
TRAIN    loss : 0.01375    f1 : 0.99157
Val    loss : 0.14901    f1 : 0.83542
epoch : 38/100    time : 54s/3366s
TRAIN    loss : 0.00764    f1 : 0.99407
Val    loss : 0.14495    f1 : 0.80874
epoch : 39/100    time : 55s/3373s
TRAIN    loss : 0.00960    f1 : 0.99058
Val    loss : 0.16501    f1 : 0.81905
epoch : 40/100    time : 55s/3287s
TRAIN    loss : 0.01359    f1 : 0.98940
Val    loss : 0.14813

epoch : 49/100    time : 56s/2848s
TRAIN    loss : 0.02349    f1 : 0.98831
Val    loss : 0.26322    f1 : 0.79326
epoch : 50/100    time : 53s/2661s
TRAIN    loss : 0.01066    f1 : 0.98845
Val    loss : 0.20192    f1 : 0.81267
epoch : 51/100    time : 58s/2839s
TRAIN    loss : 0.02519    f1 : 0.97834
Val    loss : 0.24916    f1 : 0.75607
epoch : 52/100    time : 54s/2592s
TRAIN    loss : 0.03072    f1 : 0.96966
Val    loss : 0.18119    f1 : 0.78709
epoch : 53/100    time : 54s/2550s
TRAIN    loss : 0.01845    f1 : 0.97295
Val    loss : 0.20235    f1 : 0.75240
epoch : 54/100    time : 53s/2454s
TRAIN    loss : 0.02194    f1 : 0.98191
Val    loss : 0.18281    f1 : 0.80024
-----------------SAVE:55 epoch----------------
epoch : 55/100    time : 57s/2565s
TRAIN    loss : 0.00925    f1 : 0.99361
Val    loss : 0.20515    f1 : 0.82699
epoch : 56/100    time : 54s/2373s
TRAIN    loss : 0.02150    f1 : 0.97570
Val    loss : 0.18432    f1 : 0.77332
epoch : 57/100    time : 55s/2349s
TRAIN    loss 

epoch : 16/100    time : 55s/4591s
TRAIN    loss : 0.06421    f1 : 0.93001
Val    loss : 0.20594    f1 : 0.71795
epoch : 17/100    time : 56s/4614s
TRAIN    loss : 0.06039    f1 : 0.93674
Val    loss : 0.27679    f1 : 0.75842
-----------------SAVE:18 epoch----------------
epoch : 18/100    time : 54s/4444s
TRAIN    loss : 0.05362    f1 : 0.94809
Val    loss : 0.19344    f1 : 0.77759
-----------------SAVE:19 epoch----------------
epoch : 19/100    time : 55s/4429s
TRAIN    loss : 0.04878    f1 : 0.96229
Val    loss : 0.18492    f1 : 0.77973
epoch : 20/100    time : 57s/4556s
TRAIN    loss : 0.03979    f1 : 0.96559
Val    loss : 0.22037    f1 : 0.75125
epoch : 21/100    time : 56s/4390s
TRAIN    loss : 0.04639    f1 : 0.94884
Val    loss : 0.22804    f1 : 0.73957
-----------------SAVE:22 epoch----------------
epoch : 22/100    time : 55s/4286s
TRAIN    loss : 0.03536    f1 : 0.96560
Val    loss : 0.18182    f1 : 0.79652
epoch : 23/100    time : 56s/4329s
TRAIN    loss : 0.03897    f1 : 0

epoch : 86/100    time : 57s/797s
TRAIN    loss : 0.02278    f1 : 0.98683
Val    loss : 0.22432    f1 : 0.77066
epoch : 87/100    time : 54s/699s
TRAIN    loss : 0.00734    f1 : 0.99594
Val    loss : 0.22004    f1 : 0.81607
epoch : 88/100    time : 54s/645s
TRAIN    loss : 0.01021    f1 : 0.99370
Val    loss : 0.24329    f1 : 0.75369
epoch : 89/100    time : 56s/619s
TRAIN    loss : 0.01838    f1 : 0.98170
Val    loss : 0.25865    f1 : 0.76142
epoch : 90/100    time : 54s/535s
TRAIN    loss : 0.01520    f1 : 0.98636
Val    loss : 0.18877    f1 : 0.77839
epoch : 91/100    time : 54s/488s
TRAIN    loss : 0.00372    f1 : 0.99710
Val    loss : 0.24814    f1 : 0.77677
epoch : 92/100    time : 55s/443s
TRAIN    loss : 0.00490    f1 : 0.99353
Val    loss : 0.20452    f1 : 0.80234
epoch : 93/100    time : 55s/382s
TRAIN    loss : 0.00246    f1 : 0.99868
Val    loss : 0.21094    f1 : 0.76786


In [29]:
pred_ensemble = []
batch_size = 16
# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

for i in range(5):
  model_test = Network(mode = 'test').to(device)
  model_test.load_state_dict(torch.load((path+'best_model_{}.pth'.format(i)))['state_dict'])
  model_test.eval()
  pred_prob = []
  with torch.no_grad():
      for batch in (test_loader):
          x = torch.tensor(batch[0], dtype = torch.float32, device = device)
          with torch.cuda.amp.autocast():
              pred = model_test(x)
              pred_prob.extend(pred.detach().cpu().numpy())
      pred_ensemble.append(pred_prob)

In [30]:
len(pred_ensemble)

5

In [31]:
pred = (np.array(pred_ensemble[0])+ np.array(pred_ensemble[1])+ np.array(pred_ensemble[3]) + np.array(pred_ensemble[4]) )/4
f_pred = np.array(pred).argmax(1).tolist()

In [32]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

In [33]:
submission = pd.read_csv(path + "open/sample_submission.csv")

submission["label"] = f_result

submission

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


In [34]:
submission.to_csv(path + "densenet121_norm_epoch100.csv", index = False)

In [35]:
# d2b19ece8c7374053ee1fd80cfe419ddfc640c01f9ebe4cbd5caeb9f1906974a

from dacon_submit_api import dacon_submit_api 

result = dacon_submit_api.post_submission_file(
'/home/densenet121_norm_epoch100.csv', 
'd2b19ece8c7374053ee1fd80cfe419ddfc640c01f9ebe4cbd5caeb9f1906974a', 
'235894', 
'mean', 
'densenet121_BS16' )

# https://www.dacon.io/competitions/official/235894/overview/rules

{'isSubmitted': True, 'detail': 'Success'}


### 모델 학습

사전 학습 모델의 성능 파악을 할 때 Fold 학습은 실행 시간이 오래걸려서 fold를 나누지 않은 데이터에 대해서 학습을 진행하고 성능을 비교하였습니다.

In [18]:
batch_size = 16
epochs = 30

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train')
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [19]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

model = Network().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay = 1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 

batch_size = 16
epochs = 30

best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    
    train_f1 = score_function(train_y, train_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')

### 추론

In [80]:
model.eval()
f_pred = []
pred_prob = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
            pred_prob.extend(pred.detach().cpu().numpy())
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [ ]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

### 제출물 생성

In [ ]:
submission = pd.read_csv(path + "open/sample_submission.csv")

submission["label"] = f_result

submission

In [ ]:
submission.to_csv(path + "VGG16_norm.csv", index = False)